In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-urbanshift' 
aws_s3_dir = 'https://'+bucket_name+'.s3.eu-west-3.amazonaws.com'

# Load data

In [ ]:
# read Tree Mosaic Land
tml = ee.ImageCollection('projects/wri-datalab/TML')
tml_tree_cover = tml.reduce(ee.Reducer.mean()).rename('b1')

In [ ]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir+'/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

# Extract TML by city

In [ ]:
for i in range(20,21): #len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name'] + '-' + boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = aws_s3_dir+'/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # Download ee.Image for TML as GeoTIFF
    geemap.ee_export_image_to_drive(
        tml_tree_cover.toByte(), # use toByte() to reduce file size
        description = boundary_id + '-TML-TreeCover',
        folder='data', 
        scale=10, #10 or 50
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )

In [ ]:
# connect to s3
# aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
for i in range(1, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id +'-TML-TreeCover.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/tree_cover/tree_mosaic_land/v_0/'+ boundary_id + '-TML-tree_cover-2020.tif',
                               ExtraArgs={'ACL':'public-read'})